# Example 01 - Simple usage of PIVsuite

This example demonstrates the simplest possible use of PIVsuite for obtaining the velocity field from a pair of images. For running this example, you need Python with numpy, matplotlib, and the PIVsuite Python implementation.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import sys
import os



In [5]:

# Add the parent directory to the path so we can import the modules
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))


In [6]:

from pivSuite.piv_parameters import PIVParameters
from pivSuite.pivAnalyzeImagePair import piv_analyze_image_pair
from pivSuite.pivQuiver import pivQuiver

## Before running PIVsuite

Initialize the parameters for PIV algorithm (such as interrogation area size) and prepare for storing results.

In [7]:
# Initialize variables
piv_par = PIVParameters()  # variable for settings
piv_data = {}              # variable for storing results

Define image pair to be treated. The best way is to store image's paths and filenames:

In [8]:
im1 = '../Data/Test von Karman/PIVlab_Karman_01.bmp'
im2 = '../Data/Test von Karman/PIVlab_Karman_02.bmp'
# NOTE: Use slash symbol (/) as path separator, both on Windows and Unix-based machines.

These images (kindly provided by William Thielicke) show a flow around a cylinder.

The velocity field should not be evaluated inside the cylinder. For this reason a _mask_ is provided, and particle motion is not searched inside masked area. The mask is defined by an image file (black-and-white format) in this example. The masking image has pixel value of 0 for masked area (inside the cylinder), and 1 for unmasked area (flow region). Store the mask to piv_par variable.

In [9]:
im_mask = '../Data/Test von Karman/PIVlab_Karman_mask.png'  # Again, use slash (/) as path separator.
piv_par.imMask1 = im_mask      # mask for first image
piv_par.imMask2 = im_mask      # mask for the second image

## Default settings for PIV algorithm

PIVsuite has many settings. The default settings will work well in most cases. The PIVParameters class already initializes with default values, so we don't need to explicitly call a function for this.

## Run the analysis

For PIV analysis of an image pair, execute the following command (treatment might take from seconds to a few minutes):

In [11]:
# Load the images
import imageio.v3 as io

# Load the images
image1 = io.imread(im1)
image2 = io.imread(im2)

# Run the PIV analysis
piv_data = piv_analyze_image_pair(image1, image2, piv_data, piv_par)

FileNotFoundError: No such file: '/home/user/Documents/repos/OpenOpticalFlow_PIV_v1/src/Data/Test von Karman/PIVlab_Karman_01.bmp'

## Visualize results

The variable piv_data contains now results of the PIV analysis. Use the piv_quiver function to show the velocity field. We will show a color background with velocity magnitude, overlaid by velocity vectors (_quiver plot_):

In [ ]:
plt.figure(figsize=(10, 8))
piv_quiver(piv_data, 
           background='Umag',        # show background with magnitude
           quiver=True,              # show quiver vectors
           quiver_color='k')         # show quiver vectors in black
plt.title('Particle displacement (px) in a flow around a cylinder')
plt.xlabel('position X (px)')
plt.ylabel('position Y (px)')
plt.colorbar(label='Velocity magnitude (px)')
plt.show()

Previous figure is too small. Show another figure, showing flow details around the cylinder:

In [ ]:
plt.figure(figsize=(10, 8))
piv_quiver(piv_data, 
           crop=[350, 850, 230, 530],  # set cropping
           background='Umag',          # show background with magnitude
           quiver=True,                # show quiver vectors
           quiver_color='k')           # show quiver vectors in black
plt.title('Particle displacement (px) in a flow around a cylinder (detail)')
plt.xlabel('position X (px)')
plt.ylabel('position Y (px)')
plt.colorbar(label='Velocity magnitude (px)')
plt.show()

## Extract data

Velocity field is stored in variable `piv_data` in the following fields:

* `X` ... horizontal position of velocity vectors (in pixels from the left of the image)
* `Y` ... vertical position of velocity vectors (in pixels from the top of the image)
* `U` ... horizontal component of the velocity (particle displacement between the two images, in pixels, positive value pointing to the right)
* `V` ... vertical component of the velocity (particle displacement between the two images, in pixels, positive value pointing downward).

Interpolate the data to show a velocity profile. In this example, a profile of U component is shown for `X = 550`:

In [ ]:
# Interpolate data for desired position
from scipy.interpolate import griddata

y = np.arange(230, 531)
x = np.full_like(y, 550)

# Prepare points for interpolation
points = np.column_stack((piv_data['X'].flatten(), piv_data['Y'].flatten()))
values = piv_data['U'].flatten()
target_points = np.column_stack((x, y))

# Interpolate
u = griddata(points, values, target_points, method='linear')

# Plot velocity profile
plt.figure(figsize=(8, 6))
plt.plot(u, y, '-b')
plt.title('U profile at X = 550 px')
plt.xlabel('particle displacement U (px)')
plt.ylabel('position Y (px)')
plt.grid(True)
plt.show()

Get a number of invalid velocity vectors.

In [ ]:
print(f"Grid points: {piv_data['N']}. Masked vectors: {piv_data['maskedN']}. "
      f"Spurious vectors: {piv_data['spuriousN']}. Computational time: {sum(piv_data['infCompTime'])}")